In [1]:
# импортируем необходимые библиотеки и классы
import numpy as np
import pandas as pd
from sklearn.model_selection import (GroupKFold, 
                                     TimeSeriesSplit)

In [2]:
# записываем CSV-файл в объект DataFrame
toy_data = pd.read_csv('Data/toy_example.csv', sep=';')
toy_data

,Client,Date,Transaction,Status
0,Petrov,01.01.2011,12000,0
1,Petrov,01.01.2013,24000,0
2,Petrov,01.01.2012,34000,0
3,Ivanov,01.01.2013,10000,1
4,Ivanov,01.01.2012,45000,1
5,Ivanov,01.01.2011,15000,1
6,Sidorov,01.01.2012,18000,1
7,Sidorov,01.01.2011,69000,1
8,Sidorov,01.01.2013,80000,1


In [3]:
# преобразовываем в формат даты
toy_data['Date'] = pd.to_datetime(toy_data['Date'], format='%d.%m.%Y')
# сортируем по дате и перезадаем индекс без вставки
# индекса в качестве отдельного столбца
toy_data = toy_data.sort_values(by=['Date']).reset_index(drop=True)
toy_data

,Client,Date,Transaction,Status
0,Petrov,2011-01-01,12000,0
1,Ivanov,2011-01-01,15000,1
2,Sidorov,2011-01-01,69000,1
3,Petrov,2012-01-01,34000,0
4,Ivanov,2012-01-01,45000,1
5,Sidorov,2012-01-01,18000,1
6,Petrov,2013-01-01,24000,0
7,Ivanov,2013-01-01,10000,1
8,Sidorov,2013-01-01,80000,1


In [4]:
# пишем класс GroupKFoldPlusTimeSeriesSplit, выполняющий 
# проверку GroupKFold + TimeSeriesSplit
class GroupKFoldPlusTimeSeriesSplit():
    """
    Выполняет перекрестную проверку на основе классов
    GroupKFold и TimeSeriesSplit
    
    Параметры
    ----------    
    group_n_splits:
        Количество разбиений для GroupKFold.
    time_n_splits:
        Количество разбиений для TimeSeriesSplit.
    """
    
    def __init__(self, group_n_splits, time_n_splits):
        self.group_n_splits = group_n_splits
        self.time_n_splits = time_n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.group_n_splits
    
    def split(self, X, y=None, groups=None):
        group_kfold = GroupKFold(n_splits=self.group_n_splits)
        for train_index, test_index in group_kfold.split(
            X, y, groups=groups):
            tscv = TimeSeriesSplit(n_splits=self.time_n_splits)
            for train_index_t, test_index_t in tscv.split(X, y):
                yield (np.intersect1d(train_index, train_index_t), 
                       np.intersect1d(test_index, test_index_t))         

In [5]:
# создаем экземпляр класса GroupKFoldPlusTimeSeriesSplit
groupkfold_time_cv = GroupKFoldPlusTimeSeriesSplit(
    group_n_splits=3, time_n_splits=2)
# создаем массив признаков и массив меток
X_toy = toy_data.drop(columns=['Status'])
y_toy = toy_data['Status']
# задаем идентификатор меток
groups = toy_data['Client']

# взглянем на индексы наблюдений, попавших 
# в обучающий и тестовый наборы
for train_index, test_index in groupkfold_time_cv.split(
    X_toy, y_toy, groups=groups):
    print("---------")
    print("TRAIN:", train_index)
    print("TEST:", test_index)
    X_train = X_toy.iloc[train_index]
    X_test = X_toy.iloc[test_index]
    print("---")
    print("X_train\n", X_train)
    print("X_test\n", X_test)

---------
TRAIN: [0 1]
TEST: [5]
---
X_train
    Client       Date  Transaction
0  Petrov 2011-01-01        12000
1  Ivanov 2011-01-01        15000
X_test
     Client       Date  Transaction
5  Sidorov 2012-01-01        18000
---------
TRAIN: [0 1 3 4]
TEST: [8]
---
X_train
    Client       Date  Transaction
0  Petrov 2011-01-01        12000
1  Ivanov 2011-01-01        15000
3  Petrov 2012-01-01        34000
4  Ivanov 2012-01-01        45000
X_test
     Client       Date  Transaction
8  Sidorov 2013-01-01        80000
---------
TRAIN: [1 2]
TEST: [3]
---
X_train
     Client       Date  Transaction
1   Ivanov 2011-01-01        15000
2  Sidorov 2011-01-01        69000
X_test
    Client       Date  Transaction
3  Petrov 2012-01-01        34000
---------
TRAIN: [1 2 4 5]
TEST: [6]
---
X_train
     Client       Date  Transaction
1   Ivanov 2011-01-01        15000
2  Sidorov 2011-01-01        69000
4   Ivanov 2012-01-01        45000
5  Sidorov 2012-01-01        18000
X_test
    Client       